# Libraries and Data

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install openai==0.28

!pip install langchain_community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existi

In [ ]:
import os
import re
import json
import pickle
import requests
import numpy as np
import pandas as pd
from collections import Counter
from urllib.parse import urlparse

# Import necessary sklearn modules for machine learning
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Import LangChain modules for building language model chains and agents
from langchain import OpenAI, LLMChain, PromptTemplate, SerpAPIWrapper
from langchain.agents import (AgentExecutor, AgentType, LLMSingleActionAgent, Tool, initialize_agent, load_tools)
from langchain.callbacks.manager import (AsyncCallbackManagerForToolRun, CallbackManagerForToolRun)
from langchain.chains import LLMChain, SequentialChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import BaseTool
from langchain.utilities import (GoogleSearchAPIWrapper, GoogleSerperAPIWrapper, WikipediaAPIWrapper)
from langchain.chat_models import ChatOpenAI

import json
from typing import List
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
open_ai_key=os.environ["OPENAI_API_KEY"]


# Dev News

In [ ]:
#Development Dataset

import os
import pandas as pd

# Path to your Colab folder containing the .txt files
folder_path = '/content/drive/MyDrive/UNED/SemEval/EN/subtask-1-documents_test/'


# List all .txt files in the folder
txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]

# Prepare a list to store data
data = []

# Read each file and store its name and content
for file_name in txt_files:
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        data.append({'id': file_name, 'news': content})

# Create a DataFrame
df_dev_news = pd.DataFrame(data)

# Display the DataFrame
df_dev_news



,id,news
0,RU_TEST_00012.txt,Full-Spectrum Psyop: US Whips Up Fear of Russi...
1,CC_TEST_00053.txt,Climate fearmongering based on corrupted data?...
2,RU_TEST_00061.txt,Russia Ukraine conflict Russia’s Medvedev says...
3,EN_CC_200164.txt,Dirty Business: WEF Said Reduce Laundry to Sav...
4,RU_TEST_00028.txt,"Rushing Into Catastrophe, by Paul Craig Robert..."
...,...,...
58,RU_TEST_00013.txt,My Take | Nato barbarians are expanding and ga...
59,RU_TEST_00085.txt,Ukraine’s Massive Money Laundering Scam \n\n B...
60,RU_TEST_00067.txt,How ruthless Russian oligarchs are ‘MURDERING ...
61,CC_TEST_00017.txt,Data Refutes Media’s ‘Attribution’ Con That Cl...


# Dev Entities

In [ ]:
import pandas as pd
from typing import List

# Path to the annotations file
annotations_file_path = '/content/drive/MyDrive/UNED/SemEval/EN/subtask-1-entity-mentions_test.txt'  # Replace with your folder path

# Role hierarchy mapping
role_hierarchy = {
    "Protagonist": ["Guardian", "Martyr", "Peacemaker", "Rebel", "Underdog", "Virtuous"],
    "Antagonist": ["Instigator", "Conspirator", "Tyrant", "Foreign Adversary", "Traitor",
                   "Spy", "Saboteur", "Corrupt", "Incompetent", "Terrorist", "Deceiver", "Bigot"],
    "Innocent": ["Forgotten", "Exploited", "Victim", "Scapegoat"]
}

# Reverse mapping for sub-role lookup
sub_role_to_role = {sub_role: role for role, sub_roles in role_hierarchy.items() for sub_role in sub_roles}

# Read the annotations file
annotations_data = []
with open(annotations_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('\t')
        origin_document = parts[0]
        entity = parts[1]

        # Parse Location as List[int]
        location: List[int] = list(map(int, parts[2:4]))  # Ensure the location values are integers

        roles = parts[4:]  # Store roles as a list

        # Initialize role and sub-role
        role = None
        sub_roles = []

        # Determine the role and collect all sub-roles
        for r in roles:
            if r in sub_role_to_role:  # Check if the role matches any sub-role
                sub_roles.append(r)  # Collect sub-role
                role = sub_role_to_role[r]  # Assign the corresponding role

        annotations_data.append({
            'id': origin_document,
            'Entity': entity,
            'Location': location,  # Explicitly a List[int]
            'Role': role,
            'Sub-role': sub_roles  # Store sub-roles as a list
        })

# Create a DataFrame
df_roles = pd.DataFrame(annotations_data)

# Verify Location format is List[int]
def is_list_of_int(entry) -> bool:
    return isinstance(entry, list) and all(isinstance(i, int) for i in entry)

# Add validation column
df_roles['Location_Valid'] = df_roles['Location'].apply(is_list_of_int)

# Print rows with invalid locations, if any
invalid_locations = df_roles[~df_roles['Location_Valid']]
if not invalid_locations.empty:
    print("Invalid Location entries:")
    print(invalid_locations)
else:
    print("All Location entries are valid.")

# Display the DataFrame
print(df_roles)


All Location entries are valid.
                    id          Entity      Location  Role Sub-role  \
0    CC_TEST_00063.txt  Climate Action    [787, 800]  None       []   
1    RU_TEST_00021.txt        the West    [167, 174]  None       []   
2    RU_TEST_00021.txt            NATO    [180, 183]  None       []   
3    RU_TEST_00013.txt     South Korea    [220, 230]  None       []   
4    RU_TEST_00013.txt           Japan    [233, 237]  None       []   
..                 ...             ...           ...   ...      ...   
230  RU_TEST_00045.txt            Iran  [2582, 2585]  None       []   
231  RU_TEST_00045.txt     North Korea  [2591, 2601]  None       []   
232  RU_TEST_00045.txt           Hamas  [2821, 2825]  None       []   
233  RU_TEST_00045.txt       Hezbollah  [2828, 2836]  None       []   
234  RU_TEST_00045.txt     the Houthis  [2843, 2853]  None       []   

     Location_Valid  
0              True  
1              True  
2              True  
3              True  
4    

# Dev Annotations

In [ ]:
import pandas as pd
from typing import List

# Path to the annotations file
annotations_file_path = '/content/drive/MyDrive/UNED/SemEval/EN/subtask-1-annotations.txt'  # Replace with your folder path

# Role hierarchy mapping
role_hierarchy = {
    "Protagonist": ["Guardian", "Martyr", "Peacemaker", "Rebel", "Underdog", "Virtuous"],
    "Antagonist": ["Instigator", "Conspirator", "Tyrant", "Foreign Adversary", "Traitor",
                   "Spy", "Saboteur", "Corrupt", "Incompetent", "Terrorist", "Deceiver", "Bigot"],
    "Innocent": ["Forgotten", "Exploited", "Victim", "Scapegoat"]
}

# Reverse mapping for sub-role lookup
sub_role_to_role = {sub_role: role for role, sub_roles in role_hierarchy.items() for sub_role in sub_roles}

# Read the annotations file
annotations_data = []
with open(annotations_file_path, 'r', encoding='utf-8') as file:
    for line in file:
        parts = line.strip().split('\t')
        origin_document = parts[0]
        entity = parts[1]

        # Parse Location as List[int]
        location: List[int] = list(map(int, parts[2:4]))  # Ensure the location values are integers

        roles = parts[4:]  # Store roles as a list

        # Initialize role and sub-role
        role = None
        sub_roles = []

        # Determine the role and collect all sub-roles
        for r in roles:
            if r in sub_role_to_role:  # Check if the role matches any sub-role
                sub_roles.append(r)  # Collect sub-role
                role = sub_role_to_role[r]  # Assign the corresponding role

        annotations_data.append({
            'id': origin_document,
            'Entity': entity,
            'Location': location,  # Explicitly a List[int]
            'Role': role,
            'Sub-role': sub_roles  # Store sub-roles as a list
        })

# Create a DataFrame
df_annotations = pd.DataFrame(annotations_data)

# Verify Location format is List[int]
def is_list_of_int(entry) -> bool:
    return isinstance(entry, list) and all(isinstance(i, int) for i in entry)

# Add validation column
df_annotations['Location_Valid'] = df_annotations['Location'].apply(is_list_of_int)

# Print rows with invalid locations, if any
invalid_locations = df_annotations[~df_annotations['Location_Valid']]
if not invalid_locations.empty:
    print("Invalid Location entries:")
    print(invalid_locations)
else:
    print("All Location entries are valid.")

# Display the DataFrame
print(df_annotations)

All Location entries are valid.
                      id                Entity      Location         Role  \
0   EN_UA_DEV_100012.txt            Washington  [1441, 1450]   Antagonist   
1   EN_UA_DEV_100012.txt             Americans  [2397, 2405]   Antagonist   
2       EN_CC_200053.txt           Rishi Sunak      [85, 95]   Antagonist   
3       EN_CC_200040.txt  Extinction Rebellion    [189, 208]   Antagonist   
4       EN_CC_200070.txt         Eric Garcetti     [89, 101]   Antagonist   
..                   ...                   ...           ...          ...   
86  EN_UA_DEV_100002.txt              Zelensky    [657, 664]   Antagonist   
87  EN_UA_DEV_100002.txt               Ukraine    [672, 678]     Innocent   
88  EN_UA_DEV_100002.txt                  West    [725, 728]   Antagonist   
89  EN_UA_DEV_100002.txt                  NATO  [1027, 1030]   Antagonist   
90  EN_UA_DEV_100002.txt                Russia  [1200, 1205]  Protagonist   

                            Sub-role  Locat

# Evaluation Function

In [ ]:
def normalize_subroles(subrole):
    """Ensures subroles are stored as lists and handles NaN values."""
    if isinstance(subrole, str):
        return [subrole]
    if isinstance(subrole, list):
        return subrole
    return []  # If NaN or unexpected type, return an empty list

def calculate_metrics(df_annotations, df_roles):
    """Calculates accuracy metrics for role and subrole classification."""

    # Normalize subroles and handle NaN values
    df_annotations['Sub-role'] = df_annotations['Sub-role'].apply(normalize_subroles)
    df_roles['Sub-role'] = df_roles['Sub-role'].apply(normalize_subroles)

    # Ensure 'Location' is hashable and handle NaN values
    df_annotations['Location'] = df_annotations['Location'].apply(lambda x: tuple(x) if isinstance(x, list) else ())
    df_roles['Location'] = df_roles['Location'].apply(lambda x: tuple(x) if isinstance(x, list) else ())

    # Merge based on common identifiers while preserving all annotations
    df_merged = pd.merge(df_annotations, df_roles, on=['id', 'Entity', 'Location'], how="left", suffixes=('_gold', '_pred'))

    # Fill missing predictions with empty values
    df_merged['Role_pred'] = df_merged['Role_pred'].fillna("Unknown")
    df_merged['Sub-role_pred'] = df_merged['Sub-role_pred'].apply(lambda x: x if isinstance(x, list) else [])

    # Calculate Main Role Accuracy
    main_role_accuracy = (df_merged['Role_gold'] == df_merged['Role_pred']).mean()

    # Calculate Exact Match for Subroles (unordered match)
    exact_match_subrole = df_merged.apply(lambda x: set(x['Sub-role_gold']) == set(x['Sub-role_pred']), axis=1).mean()

    return {
        "Main Role Accuracy": main_role_accuracy,
        "Exact Match for Subroles": exact_match_subrole
    }


# Architectural Systems

# 1-Input Preprocessing Using a Hierarchical Signal Approach

In [ ]:
# Define schemas
class RoleAnalysisResult(BaseModel):
    role: str = Field(description="Primary role of the entity without adding anything else")


class SubroleAnalysisResult(BaseModel):
    subrole: str = Field(description="Sub-role of the entity without adding anything else")

class SummaryResult(BaseModel):
    summary: str = Field(description="A concise summary of the article's main focus")

# Base class for agents
class BaseAgent:
    def __init__(self, model: str, temperature: float = 0):
        self.llm = ChatOpenAI(model_name=model, temperature=temperature)


class ArticleSummaryAgent (BaseAgent):
    def process_results(self, article:str, entity:str) -> SummaryResult:


        # Prompt for summarization
        prompt = (
            f"Task:\n"
            f"You are an expert summarizer. Your task is to provide a concise summary of the following article refering to the {entity}:\n\n"
            f"Article:\n{article}\n\n"
            f"Output Format:\n"
            f"Summary: [Provide the article's summary taking into consideration the role of the entity in it]"
        )

        # Query the LLM
        response = self.llm.invoke([HumanMessage(content=prompt)])
        summary = response.content.strip()

        # Validate that a summary was extracted
        if not summary.startswith("Summary:"):
            raise ValueError(f"Invalid response format: {response.content}")

        # Extract the summary text
        summary_text = summary.split(":", 1)[1].strip()

        return SummaryResult(summary=summary_text)

# Role Discrimination Agent
class RoleAgent_UA(BaseAgent):
    def process_results(self, entity: str, context: str, summary:str) -> RoleAnalysisResult:
        """
        Discriminate between Antagonist, Protagonist, and Innocent roles.
        """
        prompt = (
      f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the role of an entity {entity} based on its portrayal, actions, or impact in a news article {summary}.
          of an article related to the Ukrainian conflict, you will be provide with the paragraph {context} where the entity is mentioned.
          Categorize the entity into one of the following roles: Protagonist, Antagonist, or Innocent, and provide a justification grounded in the taxonomy definitions.
          Instructions:
          1.Roles:

              Protagonist: Entities portrayed as positively contributing to Ukraine's sovereignty, defending justice, protecting civilians, or promoting peace. They are often champions of resistance, protectors, or advocates for justice within the conflict.
              Antagonist: Entities portrayed as instigating harm, causing conflict, oppressing others, or undermining peace and sovereignty in Ukraine. Their actions are seen as harmful, unethical, or disruptive to stability in the region.
              Innocent: Entities portrayed as neutral, passive, or victims of the Ukrainian conflict. They are often affected by war, exploited, or unjustly blamed for outcomes beyond their control.



          2.Output Format:

              Role: [Protagonist/Antagonist/Innocent]
              Just provide the most suitable label without adding anything else—just the classification label.

          """
        )



        response = self.llm.invoke([HumanMessage(content=prompt)])
        print("Model Response:")
        print(response.content)
        role=response.content

        # Validate that a role was extracted
        if not role:
            raise ValueError("Role could not be determined. Ensure the prompt and response format are correct.")

        return RoleAnalysisResult(
            role=role,

        )


# Role Discrimination Agent
class RoleAgent_CC(BaseAgent):
    def process_results(self, entity: str, context: str, summary:str)  -> RoleAnalysisResult:
        """
        Discriminate between Antagonist, Protagonist, and Innocent roles.
        """
        prompt = (

        f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the role of an entity {entity} based on its portrayal, actions, or impact in a news article {summary}.
          of an article related to Climate Change, you will be provide with the paragraph {context} where the entity is mentioned.
          Categorize the entity into one of the following roles: Protagonist, Antagonist, or Innocent, and provide a justification grounded in the taxonomy definitions.tagonist, Antagonist, or Innocent, and provide a justification grounded in the taxonomy definitions.
        Instructions:
        1.Roles:
          Protagonist: Entities portrayed as positively contributing to climate change solutions, defending environmental values, promoting sustainability, or acting for the greater good. They are often advocates for change, protectors of the environment, or role models in the fight against climate change.
          Antagonist: Entities portrayed as obstructing climate change solutions, causing environmental harm, or undermining sustainability efforts. Their actions are seen as harmful, unethical, or counterproductive to environmental goals.
          Innocent: Entities portrayed as neutral, passive, or victims of climate change-related circumstances. They are often affected by external factors, exploited, or unfairly blamed for environmental issues.


        2.Output Format:
          Role: [Protagonist/Antagonist/Innocent]
          Just provide the most suitable label without adding anything else just the classification label

        """

      )



        response = self.llm.invoke([HumanMessage(content=prompt)])
        print("Model Response:")
        print(response.content)
        role=response.content

        # Validate that a role was extracted
        if not role:
            raise ValueError("Role could not be determined. Ensure the prompt and response format are correct.")

        return RoleAnalysisResult(
            role=role,

        )





# Protagonist Subrole Discrimination Agent
class SubroleAgentProtagonist_CC(BaseAgent):
    def process_results(self, entity: str, context: str, summary: str) -> SubroleAnalysisResult:

        prompt = (

        f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity} based on its portrayal, actions, or impact in a news article {summary}.
          of an article related to Climate Change, you will be provide with the paragraph {context} where the entity is mentioned.
          into one of the following Protagonist subroles: Guardian, Rebel, or Virtuous.

          Instructions:
          1. Subrole Definitions for Protagonists:
            - Guardian: Entities portrayed as protectors of values, communities, or the environment. Guardians take proactive measures to ensure safety, justice, and sustainability.
              - Example: Community leaders advocating for climate action, organizations safeguarding ecosystems, or individuals working to prevent deforestation.
            - Rebel: Entities portrayed as challengers to the status quo, fighting for transformative change against oppressive systems or harmful practices. Rebels are often seen as champions of justice and liberation in the context of climate change.
              - Example: Activists like Greta Thunberg or individuals chaining themselves to trees to protest deforestation.
            - Virtuous: Entities depicted as noble, fair, and morally upright, serving as role models in the fight against climate change. They often uphold ethical standards and inspire others through their integrity.
              - Example: Leaders promoting environmental sustainability, activists advocating for renewable energy, or judges upholding climate justice.


          2. Output Format:

            - Subrole: [Guardian/Rebel/Virtuous]
            - Just provide the most suitable label without adding anything else just the classification label


        """

        )

        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)



class SubroleAgentProtagonist_UA(BaseAgent):
    def process_results(self, entity: str, context: str, summary: str) -> SubroleAnalysisResult:
        """
        Discriminate between subroles of the Protagonist: Guardian, Martyr, Peacemaker, Rebel, Underdog, Virtuous.
        """
        prompt = (
        f"""
          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity} based on its portrayal, actions, or impact in a news article {summary}.
          of an article related to Climate Change, you will be provide with the paragraph {context} where the entity is mentioned.
          into one of the following Protagonist subroles: Guardian, Martyr, Peacemaker, Rebel, Underdog, or Virtuous.


            Instructions:

            1. Subrole Definitions for Protagonists:
              - Guardian: Entities portrayed as protectors of values, people, or national sovereignty. Guardians actively ensure safety, justice, or defense in the conflict.
                - Example: Ukrainian Armed Forces defending their territory from aggression.
              - Martyr: Entities depicted as sacrificing themselves for a greater cause, such as freedom, justice, or peace. Their actions often evoke admiration for their selflessness.
                - Example: Civilians risking their lives to protect others or soldiers dying in battle for their country.
              - Peacemaker: Individuals or entities advocating for harmony and working toward diplomatic resolutions or mediation to end the conflict.
                - Example: International diplomats brokering peace deals or local leaders working to reduce tensions.
              - Rebel: Entities challenging oppression or an aggressive force, often fighting for liberation or justice. Rebels stand against the status quo to drive meaningful change.
                - Example: Partisans resisting occupation or individuals fighting against oppressive regimes.
              - Underdog: Entities in a disadvantaged position but striving against great odds to overcome challenges. Their resilience and determination often inspire others.
                - Example: Small towns resisting occupation or individuals leading grassroots resistance efforts.
              - Virtuous: Entities depicted as morally upright, acting with integrity and fairness. They often serve as role models or symbols of justice.
                - Example: Ukrainian leaders advocating for international support and upholding democratic values.

            2. Output Format:
              - Subrole: [Guardian/Martyr/Peacemaker/Rebel/Underdog/Virtuous]
              - Just provide the most suitable label without adding anything else just the classification label


        """
      )
        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)



# Antagonist Subrole Discrimination Agent
class SubroleAgentAntagonist_CC(BaseAgent):
    def process_results(self, entity: str, context: str, summary: str) -> SubroleAnalysisResult:
        """
        Discriminate between subroles of the Antagonist: Instigator, Conspirator, Tyrant, Foreign Adversary, Traitor,
        Spy, Saboteur, Corrupt, Incompetent, Terrorist, Deceiver, Bigot.
        """
        prompt = (
        f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity} based on its portrayal, actions, or impact in a news article {summary}.
          of an article related to Climate Change, you will be provide with the paragraph {context} where the entity is mentioned.
          you have to selec the most adequate subrole for the Antagonist role: Instigator, Conspirator, Tyrant, Traitor, Saboteur, Corrupt, Incompetent, or Deceiver.

            Instructions:

            1. Subrole Definitions for Antagonists:
              - Instigator: Entities portrayed as initiating tension, unrest, or harmful environmental activities.
                - Example: Lobbyists inciting public opposition to renewable energy projects or organizations fueling deforestation for profit.
              - Conspirator: Entities involved in covert or deceptive activities to undermine environmental regulations or policies.
                - Example: Corporations conspiring to bypass emission standards or falsify environmental reports for financial gain.
              - Tyrant: Entities abusing power to exploit resources, suppress environmental initiatives, or oppress affected communities.
                - Example: Politicians obstructing climate reforms or companies exploiting vulnerable communities for resource extraction.
              - Traitor: Entities betraying commitments to environmental causes, breaking promises, or siding with harmful agendas.
                - Example: Nations pledging to reduce emissions but secretly expanding fossil fuel projects.
              - Saboteur: Entities actively sabotaging systems, processes, or policies designed to protect the environment.
                - Example: Activists destroying clean energy projects or insiders derailing renewable energy initiatives.
              - Corrupt: Entities engaging in unethical or illegal activities, prioritizing personal or financial gain over environmental well-being.
                - Example: Officials accepting bribes to approve environmentally harmful projects.
              - Incompetent: Entities harming environmental efforts due to ignorance, poor decision-making, or lack of skill.
                - Example: Leaders mismanaging climate funds or implementing ineffective policies that worsen environmental crises.
              - Deceiver: Entities spreading misinformation or manipulating public perception about climate change for their benefit.
                - Example: Media outlets downplaying climate crises or corporations misleading consumers with greenwashing campaigns.


            2. Output Format:
              - Subrole: [Instigator/Conspirator/Tyrant/Traitor/Saboteur/Corrupt/Incompetent/Deceiver]
              - Just provide the label without adding anything else just the classification label


        """
    )


        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)

# Antagonist Subrole Discrimination Agent
class SubroleAgentAntagonist_UA(BaseAgent):
    def process_results(self, entity: str, context: str, summary: str) -> SubroleAnalysisResult:
        """
        Discriminate between subroles of the Antagonist: Instigator, Conspirator, Tyrant, Foreign Adversary, Traitor,
        Spy, Saboteur, Corrupt, Incompetent, Terrorist, Deceiver, Bigot.
        """
        prompt = (
          f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity} based on its portrayal, actions, or impact in a news article {summary}.
          of an article related to the Ukranian Conflict, you will be provide with the paragraph {context} where the entity is mentioned.
          Choose the most adequate subrole for the Antagonist role: Instigator, Conspirator, Tyrant, Foreign Adversary, Traitor, Spy, Saboteur, Corrupt, Incompetent, Terrorist, Deceiver, or Bigot.

            Instructions:

            1. Subrole Definitions for Antagonists:
              - Instigator: Entities initiating tension, violence, or unrest in the conflict.
                - Example: Politicians using inflammatory rhetoric to incite violence or groups provoking unrest to destabilize regions.
              - Conspirator: Entities engaging in covert activities or secret plans to undermine or deceive others.
                - Example: Figures involved in espionage or secret dealings to manipulate outcomes in the conflict.
              - Tyrant: Entities abusing power to oppress others, exploit resources, or impose authoritarian control.
                - Example: Political leaders using authoritarian measures to suppress dissent or exploit civilians in occupied areas.
              - Foreign Adversary: External entities or nations acting against Ukraine’s interests, often creating geopolitical tension.
                - Example: Countries involved in military aggression or cyberattacks targeting Ukraine.
              - Traitor: Individuals betraying their country or cause by supporting adversarial actions or undermining national interests.
                - Example: Politicians or officials aiding enemy forces or leaking sensitive information.
              - Spy: Entities engaged in espionage, gathering and transmitting sensitive information to rival forces or governments.
                - Example: Individuals working covertly to leak military or strategic data.
              - Saboteur: Entities deliberately disrupting systems, infrastructure, or strategies to weaken Ukraine’s efforts.
                - Example: Insiders damaging military equipment or disrupting communications during critical operations.
              - Corrupt: Entities engaging in unethical or illegal activities, prioritizing personal or financial gain over ethical considerations.
                - Example: Officials embezzling aid funds or accepting bribes to support enemy actions.
              - Incompetent: Entities causing harm through mismanagement, ignorance, or poor decision-making.
                - Example: Leaders making disastrous strategic decisions or mishandling humanitarian efforts.
              - Terrorist: Entities engaging in acts of violence or terror to further ideological, political, or military goals.
                - Example: Groups targeting civilians with attacks to instill fear and destabilize the region.
              - Deceiver: Entities spreading misinformation, propaganda, or false narratives to manipulate public perception or gain advantage.
                - Example: Media outlets promoting false claims about the conflict or government actions.
              - Bigot: Entities engaging in discriminatory actions or rhetoric against specific groups, inciting hatred or division.
                - Example: Politicians promoting ethnic or cultural hostility to justify aggressive actions.

            2. Output Format:
              - Subrole: [Instigator/Conspirator/Tyrant/Foreign Adversary/Traitor/Spy/Saboteur/Corrupt/Incompetent/Terrorist/Deceiver/Bigot]
              - Just provide the label without adding anything else just the classification label

          """
)


        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)

# Innocent Subrole Discrimination Agent
class SubroleAgentInnocent_CC(BaseAgent):
    def process_results(self, entity: str, context: str, summary: str) -> SubroleAnalysisResult:
        """
        Discriminate between subroles: Forgotten, Exploited, Victim, Scapegoat.
        """
        prompt = (
        f"""
          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity} based on its portrayal, actions, or impact in a news article {summary}.
          of an article related to Climate Change, you will be provide with the paragraph {context} where the entity is mentioned.
          Choose the most adequate subrole for the Innocent role: Forgotten, Exploited, Victim, or Scapegoat.

            Instructions:

            1. Subrole Definitions for Innocents:
              - Forgotten: Entities portrayed as marginalized or overlooked, who do not receive the attention or support they need despite being affected by climate change.
                - Example: Indigenous populations facing discrimination and exclusion from climate discussions or displaced communities left out of international aid programs.
              - Exploited: Entities portrayed as being used for others' gain, often without consent, and with significant harm to their well-being.
                - Example: Workers subjected to poor conditions in industries contributing to deforestation or communities exploited for natural resources.
              - Victim: Entities framed as suffering harm due to circumstances beyond their control, such as physical harm (natural disasters, industrial accidents) or economic harm (poverty caused by climate-related policies).
                - Example: Farmers losing livelihoods due to droughts or victims of flooding caused by rising sea levels.
              - Scapegoat: Entities unfairly blamed for problems or failures related to climate change, often to divert attention from the real causes.
                - Example: Small communities blamed for deforestation caused by corporate interests or low-income nations held responsible for global emissions despite their minimal contribution.


            2. Output Format:
              - Subrole: [Forgotten/Exploited/Victim/Scapegoat]
              - Just provide the label without adding anything else just the classification label


        """
      )

        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)

class SubroleAgentInnocent_UA(BaseAgent):
    def process_results(self, entity: str, context: str, summary: str) -> SubroleAnalysisResult:

        prompt =(

      f"""

        You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity} based on its portrayal, actions, or impact in a news article {summary}.
          of an article related to the Ukranian Conflict, you will be provide with the paragraph {context} where the entity is mentioned.
          Choose the most adequate subrole for the Innocent role: Forgotten, Exploited, Victim, or Scapegoat.
          Instructions:


          1. Subrole Definitions for Innocents:
            - Forgotten: Entities portrayed as marginalized or overlooked, receiving little attention or support despite being severely affected by the conflict.
              - Example: Refugees fleeing war who lack access to aid, displaced individuals ignored by international organizations, or minority groups excluded from relief programs.
            - Exploited: Entities used for others’ gain during the conflict, often without consent and with harm to their well-being.
              - Example: Civilians forced into labor for occupying forces, local resources exploited by foreign powers, or communities coerced into serving military needs.
            - Victim: Entities suffering harm due to circumstances beyond their control, including physical harm (bombings, invasions, or attacks) or economic harm (sanctions, loss of livelihoods).
              - Example: Families losing their homes due to missile strikes, farmers unable to harvest crops due to war conditions, or civilians affected by economic blockades.
            - Scapegoat: Entities unfairly blamed for problems or failures in the conflict, often to divert attention from the real causes.
              - Example: Minority groups accused of undermining national unity, local communities wrongly blamed for aiding opposing forces, or political opponents targeted to deflect criticism.

          2. Output Format:
            - Subrole: [Forgotten/Exploited/Victim/Scapegoat]
            - Just provide the label without adding anything else just the classification label

""")

        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)



In [ ]:
df_roles_infe_v11=df_roles.copy()

In [ ]:
df_roles_infe_v12=df_roles.copy()

# 1.1 Majority Voting

In [ ]:

# Function to extract the paragraph containing the entity
def extract_paragraph_with_entity(news_content, location):
    """
    Extract the paragraph containing the entity from the article.
    It starts from the nearest period (.) before the start location and ends at the next period (.) after the end location.
    """
    start_index = location[0]
    end_index = location[1]

    # Find the nearest period before the start index
    before = news_content[:start_index].rfind(".") + 1  # Start after the period

    # Find the nearest period after the end index
    after = news_content[end_index:].find(".")
    after = end_index + after + 1 if after != -1 else len(news_content)  # Include the period

    # Extract the paragraph and strip extra whitespace
    paragraph = news_content[before:after].strip()
    return paragraph

# Define the process function
def process_roles_and_subroles_11(df_roles, df_news):
    # Initialize agents
    summary_agent=ArticleSummaryAgent(model="gpt-4o")
    role_agent_ua = RoleAgent_UA(model="gpt-4o")
    role_agent_cc = RoleAgent_CC(model="gpt-4o")
    protagonist_agent_ua = SubroleAgentProtagonist_UA(model="gpt-4o")
    protagonist_agent_cc = SubroleAgentProtagonist_CC(model="gpt-4o")
    antagonist_agent_ua = SubroleAgentAntagonist_UA(model="gpt-4o")
    antagonist_agent_cc = SubroleAgentAntagonist_CC(model="gpt-4o")
    innocent_agent_ua = SubroleAgentInnocent_UA(model="gpt-4o")
    innocent_agent_cc = SubroleAgentInnocent_CC(model="gpt-4o")

    # Ensure columns for Role and Sub-role exist
    if 'Role' not in df_roles.columns:
        df_roles['Role'] = None
    if 'Sub-role' not in df_roles.columns:
        df_roles['Sub-role'] = None
    if 'Summary' not in df_roles.columns:
        df_roles['Summary'] = None
    if 'Context' not in df_roles.columns:
        df_roles['Context'] = None

    # Iterate over each row in df_roles
    for index, row in df_roles.iterrows():
        # Extract the necessary information
        doc_id = row['id']
        entity = row['Entity']
        location = row['Location']

        # Determine the topic from the id
        topic = "CC" if "CC" in doc_id else "UA"
        if not topic:
            print(f"Could not determine topic for id: {doc_id}")
            continue  # Skip if the topic cannot be determined

        # Get the news content from df_news based on id
        news_row = df_news[df_news['id'] == doc_id]
        if news_row.empty:
            print(f"No matching news article for id: {doc_id}")
            continue  # Skip if no matching news article

        news_content = news_row['news'].values[0]

        # Step 1: Generate summary using ArticleSummaryAgent
        try:
            summary_result = summary_agent.process_results(article=news_content, entity=entity)
            df_roles.loc[index, 'Summary'] = summary_result.summary  # Store the summary
        except Exception as e:
            print(f"Error generating summary for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip if summary extraction fails

        # Step 2: Extract context paragraph
        try:
            context = extract_paragraph_with_entity(news_content, location)
            df_roles.loc[index, 'Context'] = context  # Store the extracted context
        except Exception as e:
            print(f"Error extracting context for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip if context extraction fails

        # Step 3: Determine the role using RoleAgent
        try:
            role_agent = role_agent_ua if topic == "UA" else role_agent_cc

            # Run 3 times and determine the most common role
            roles = [role_agent.process_results(entity=entity, context=context, summary=summary_result.summary).role for _ in range(3)]
            most_common_role = Counter(roles).most_common(1)[0][0]

            df_roles.loc[index, 'Role'] = most_common_role  # Update the Role column
        except Exception as e:
            print(f"Error processing role for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

        # Step 4: Determine the subrole (if applicable)
        try:
            if most_common_role == "Protagonist":
                subrole_agent = protagonist_agent_ua if topic == "UA" else protagonist_agent_cc
            elif most_common_role == "Antagonist":
                subrole_agent = antagonist_agent_ua if topic == "UA" else antagonist_agent_cc
            elif most_common_role == "Innocent":
                subrole_agent = innocent_agent_ua if topic == "UA" else innocent_agent_cc
            else:
                continue  # Skip if role is not recognized

            # Run 3 times and determine the most common subrole
            subroles = [subrole_agent.process_results(entity=entity, context=context, summary=summary_result.summary).subrole for _ in range(3)]
            final_subrole = Counter(subroles).most_common(1)[0][0]

            df_roles.loc[index, 'Sub-role'] = final_subrole  # Update the Sub-role column
        except Exception as e:
            print(f"Error processing subroles for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

    return df_roles


task1_pred_11 = process_roles_and_subroles_11(df_roles, df_dev_news)



In [ ]:
task1_pred_11.to_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_11.csv', index=False)

In [ ]:
task1_pred_11 = pd.read_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_11.csv')

In [ ]:
results = calculate_metrics(df_annotations, task1_pred_11)
print(results)

{'Main Role Accuracy': 0.8110236220472441, 'Exact Match for Subroles': 0.2677165354330709}


# 1.2 Without Majority Voting

In [ ]:

# Function to extract the paragraph containing the entity
def extract_paragraph_with_entity(news_content, location):
    """
    Extract the paragraph containing the entity from the article.
    It starts from the nearest period (.) before the start location and ends at the next period (.) after the end location.
    """
    start_index = location[0]
    end_index = location[1]

    # Find the nearest period before the start index
    before = news_content[:start_index].rfind(".") + 1  # Start after the period

    # Find the nearest period after the end index
    after = news_content[end_index:].find(".")
    after = end_index + after + 1 if after != -1 else len(news_content)  # Include the period

    # Extract the paragraph and strip extra whitespace
    paragraph = news_content[before:after].strip()
    return paragraph

# Define the process function
def process_roles_and_subroles_11(df_roles, df_news):
    # Initialize agents
    summary_agent=ArticleSummaryAgent(model="gpt-4o")
    role_agent_ua = RoleAgent_UA(model="gpt-4o")
    role_agent_cc = RoleAgent_CC(model="gpt-4o")
    protagonist_agent_ua = SubroleAgentProtagonist_UA(model="gpt-4o")
    protagonist_agent_cc = SubroleAgentProtagonist_CC(model="gpt-4o")
    antagonist_agent_ua = SubroleAgentAntagonist_UA(model="gpt-4o")
    antagonist_agent_cc = SubroleAgentAntagonist_CC(model="gpt-4o")
    innocent_agent_ua = SubroleAgentInnocent_UA(model="gpt-4o")
    innocent_agent_cc = SubroleAgentInnocent_CC(model="gpt-4o")

    # Ensure columns for Role and Sub-role exist
    if 'Role' not in df_roles.columns:
        df_roles['Role'] = None
    if 'Sub-role' not in df_roles.columns:
        df_roles['Sub-role'] = None
    if 'Summary' not in df_roles.columns:
        df_roles['Summary'] = None
    if 'Context' not in df_roles.columns:
        df_roles['Context'] = None

    # Iterate over each row in df_roles
    for index, row in df_roles.iterrows():
        # Extract the necessary information
        doc_id = row['id']
        entity = row['Entity']
        location = row['Location']

        # Determine the topic from the id
        topic = "CC" if "CC" in doc_id else "UA"
        if not topic:
            print(f"Could not determine topic for id: {doc_id}")
            continue  # Skip if the topic cannot be determined

        # Get the news content from df_news based on id
        news_row = df_news[df_news['id'] == doc_id]
        if news_row.empty:
            print(f"No matching news article for id: {doc_id}")
            continue  # Skip if no matching news article

        news_content = news_row['news'].values[0]

        # Step 1: Generate summary using ArticleSummaryAgent
        try:
            summary_result = summary_agent.process_results(article=news_content, entity=entity)
            df_roles.loc[index, 'Summary'] = summary_result.summary  # Store the summary
        except Exception as e:
            print(f"Error generating summary for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip if summary extraction fails

        # Step 2: Extract context paragraph
        try:
            context = extract_paragraph_with_entity(news_content, location)
            df_roles.loc[index, 'Context'] = context  # Store the extracted context
        except Exception as e:
            print(f"Error extracting context for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip if context extraction fails

        # Step 3: Determine the role using RoleAgent
        try:
            role_agent = role_agent_ua if topic == "UA" else role_agent_cc


            roles = [role_agent.process_results(entity=entity, context=context, summary=summary_result.summary).role for _ in range(1)]
            most_common_role = Counter(roles).most_common(1)[0][0]

            df_roles.loc[index, 'Role'] = most_common_role  # Update the Role column
        except Exception as e:
            print(f"Error processing role for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

        # Step 4: Determine the subrole (if applicable)
        try:
            if most_common_role == "Protagonist":
                subrole_agent = protagonist_agent_ua if topic == "UA" else protagonist_agent_cc
            elif most_common_role == "Antagonist":
                subrole_agent = antagonist_agent_ua if topic == "UA" else antagonist_agent_cc
            elif most_common_role == "Innocent":
                subrole_agent = innocent_agent_ua if topic == "UA" else innocent_agent_cc
            else:
                continue  # Skip if role is not recognized

            subroles = [subrole_agent.process_results(entity=entity, context=context, summary=summary_result.summary).subrole for _ in range(1)]
            final_subrole = Counter(subroles).most_common(1)[0][0]

            df_roles.loc[index, 'Sub-role'] = final_subrole  # Update the Sub-role column
        except Exception as e:
            print(f"Error processing subroles for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

    return df_roles


task1_pred_12 = process_roles_and_subroles_11(df_roles, df_dev_news)



In [ ]:
task1_pred_12 = pd.read_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_12.csv')

In [ ]:
task1_pred_12.to_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_12.csv', index=False)

In [ ]:
results = calculate_metrics(df_annotations, task1_pred_12)
print(results)

{'Main Role Accuracy': 0.7752808988764045, 'Exact Match for Subroles': 0.30337078651685395}


# 2- No Input Preprocessing Using a Hierarchical Signal Approach

In [ ]:

# Define schemas
class RoleAnalysisResult(BaseModel):
    role: str = Field(description="Primary role of the entity without adding anything else")


class SubroleAnalysisResult(BaseModel):
    subrole: str = Field(description="Sub-role of the entity without adding anything else")



# Base class for agents
class BaseAgent:
    def __init__(self, model: str, temperature: float = 0):
        self.llm = ChatOpenAI(model_name=model, temperature=temperature)



# Role Discrimination Agent
class RoleAgent_UA(BaseAgent):
    def process_results(self, entity: str, news: str, location:list) -> RoleAnalysisResult:
        """
        Discriminate between Antagonist, Protagonist, and Innocent roles.
        """
        prompt = (
      f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the role of an entity {entity} based on its portrayal, actions, or impact in a news article {news}.
          of an article related to the Ukrainian conflict, you will be provide with the location where the entity is mentioned {location} where the entity is mentioned.
          Categorize the entity into one of the following roles: Protagonist, Antagonist, or Innocent, and provide a justification grounded in the taxonomy definitions.
          Instructions:
          1.Roles:

              Protagonist: Entities portrayed as positively contributing to Ukraine's sovereignty, defending justice, protecting civilians, or promoting peace. They are often champions of resistance, protectors, or advocates for justice within the conflict.
              Antagonist: Entities portrayed as instigating harm, causing conflict, oppressing others, or undermining peace and sovereignty in Ukraine. Their actions are seen as harmful, unethical, or disruptive to stability in the region.
              Innocent: Entities portrayed as neutral, passive, or victims of the Ukrainian conflict. They are often affected by war, exploited, or unjustly blamed for outcomes beyond their control.



          2.Output Format:

              Role: [Protagonist/Antagonist/Innocent]
              Just provide the most suitable label without adding anything else—just the classification label.

          """
        )



        response = self.llm.invoke([HumanMessage(content=prompt)])
        print("Model Response:")
        print(response.content)
        role=response.content

        # Validate that a role was extracted
        if not role:
            raise ValueError("Role could not be determined. Ensure the prompt and response format are correct.")

        return RoleAnalysisResult(
            role=role,

        )


# Role Discrimination Agent
class RoleAgent_CC(BaseAgent):
     def process_results(self, entity: str, news: str, location:list) -> RoleAnalysisResult:
        """
        Discriminate between Antagonist, Protagonist, and Innocent roles.
        """
        prompt = (
      f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the role of an entity {entity} based on its portrayal, actions, or impact in a news article {news}.
          of an article related to Climate Change, you will be provide with the location where the entity is mentioned {location} where the entity is mentioned.
          Categorize the entity into one of the following roles: Protagonist, Antagonist, or Innocent, and provide a justification grounded in the taxonomy definitions.tagonist, Antagonist, or Innocent, and provide a justification grounded in the taxonomy definitions.
        Instructions:
        1.Roles:
          Protagonist: Entities portrayed as positively contributing to climate change solutions, defending environmental values, promoting sustainability, or acting for the greater good. They are often advocates for change, protectors of the environment, or role models in the fight against climate change.
          Antagonist: Entities portrayed as obstructing climate change solutions, causing environmental harm, or undermining sustainability efforts. Their actions are seen as harmful, unethical, or counterproductive to environmental goals.
          Innocent: Entities portrayed as neutral, passive, or victims of climate change-related circumstances. They are often affected by external factors, exploited, or unfairly blamed for environmental issues.


        2.Output Format:
          Role: [Protagonist/Antagonist/Innocent]
          Just provide the most suitable label without adding anything else just the classification label

        """

      )



        response = self.llm.invoke([HumanMessage(content=prompt)])
        print("Model Response:")
        print(response.content)
        role=response.content

        # Validate that a role was extracted
        if not role:
            raise ValueError("Role could not be determined. Ensure the prompt and response format are correct.")

        return RoleAnalysisResult(
            role=role,

        )





# Protagonist Subrole Discrimination Agent
class SubroleAgentProtagonist_CC(BaseAgent):
      def process_results(self, entity: str, news: str, location:list) -> SubroleAnalysisResult:

        prompt = (

        f"""
          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity}based on its portrayal, actions, or impact in a news article {news}.
          of an article related to Climate Change, you will be provide with the location where the entity is mentioned {location} where the entity is mentioned.
          Categorize the entity into one of the following roles: Protagonist, Antagonist, or Innocent, and provide a justification grounded in the taxonomy definitions.tagonist, Antagonist, or Innocent, and provide a justification grounded in the taxonomy definitions.
          into one of the following Protagonist subroles: Guardian, Rebel, or Virtuous.

          Instructions:
          1. Subrole Definitions for Protagonists:
            - Guardian: Entities portrayed as protectors of values, communities, or the environment. Guardians take proactive measures to ensure safety, justice, and sustainability.
              - Example: Community leaders advocating for climate action, organizations safeguarding ecosystems, or individuals working to prevent deforestation.
            - Rebel: Entities portrayed as challengers to the status quo, fighting for transformative change against oppressive systems or harmful practices. Rebels are often seen as champions of justice and liberation in the context of climate change. and inspire others through their integrity.
              - Example: Leaders promoting environmental sustainability, activists advocating for renewable energy, or judges upholding climate justice.


          2. Output Format:

            - Example: Activists like Greta Thunberg or individuals chaining themselves to trees to protest deforestation.
            - Virtuous: Entities depicted as noble, fair, and morally upright, serving as role models in the fight against climate change. They often uphold ethical standards
            - Subrole: [Guardian/Rebel/Virtuous]
            - Just provide the most suitable label without adding anything else just the classification label


        """

        )

        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)



class SubroleAgentProtagonist_UA(BaseAgent):
    def process_results(self, entity: str, news: str, location:list) -> SubroleAnalysisResult:
        """
        Discriminate between subroles of the Protagonist: Guardian, Martyr, Peacemaker, Rebel, Underdog, Virtuous.
        """
        prompt = (
        f"""
          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity}based on its portrayal, actions, or impact in a news article {news}.
          of an article related to the Ukranian Conflict, you will be provided with the location where the entity is mentioned {location} where the entity is mentioned.
          into one of the following Protagonist subroles: Guardian, Martyr, Peacemaker, Rebel, Underdog, or Virtuous.


            Instructions:

            1. Subrole Definitions for Protagonists:
              - Guardian: Entities portrayed as protectors of values, people, or national sovereignty. Guardians actively ensure safety, justice, or defense in the conflict.
                - Example: Ukrainian Armed Forces defending their territory from aggression.
              - Martyr: Entities depicted as sacrificing themselves for a greater cause, such as freedom, justice, or peace. Their actions often evoke admiration for their selflessness.
                - Example: Civilians risking their lives to protect others or soldiers dying in battle for their country.
              - Peacemaker: Individuals or entities advocating for harmony and working toward diplomatic resolutions or mediation to end the conflict.
                - Example: International diplomats brokering peace deals or local leaders working to reduce tensions.
              - Rebel: Entities challenging oppression or an aggressive force, often fighting for liberation or justice. Rebels stand against the status quo to drive meaningful change.
                - Example: Partisans resisting occupation or individuals fighting against oppressive regimes.
              - Underdog: Entities in a disadvantaged position but striving against great odds to overcome challenges. Their resilience and determination often inspire others.
                - Example: Small towns resisting occupation or individuals leading grassroots resistance efforts.
              - Virtuous: Entities depicted as morally upright, acting with integrity and fairness. They often serve as role models or symbols of justice.
                - Example: Ukrainian leaders advocating for international support and upholding democratic values.

            2. Output Format:
              - Subrole: [Guardian/Martyr/Peacemaker/Rebel/Underdog/Virtuous]
              - Just provide the most suitable label without adding anything else just the classification label


        """
      )
        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)



# Antagonist Subrole Discrimination Agent
class SubroleAgentAntagonist_CC(BaseAgent):
    def process_results(self, entity: str, news: str, location:list) -> SubroleAnalysisResult:
        """
        Discriminate between subroles of the Antagonist: Instigator, Conspirator, Tyrant, Foreign Adversary, Traitor,
        Spy, Saboteur, Corrupt, Incompetent, Terrorist, Deceiver, Bigot.
        """
        prompt = (
        f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity}based on its portrayal, actions, or impact in a news article {news}.
          of an article related to Climate Change, you will be provide with the location where the entity is mentioned {location}.
          you have to selec the most adequate subrole for the Antagonist role: Instigator, Conspirator, Tyrant, Traitor, Saboteur, Corrupt, Incompetent, or Deceiver.

            Instructions:

            1. Subrole Definitions for Antagonists:
              - Instigator: Entities portrayed as initiating tension, unrest, or harmful environmental activities.
                - Example: Lobbyists inciting public opposition to renewable energy projects or organizations fueling deforestation for profit.
              - Conspirator: Entities involved in covert or deceptive activities to undermine environmental regulations or policies.
                - Example: Corporations conspiring to bypass emission standards or falsify environmental reports for financial gain.
              - Tyrant: Entities abusing power to exploit resources, suppress environmental initiatives, or oppress affected communities.
                - Example: Politicians obstructing climate reforms or companies exploiting vulnerable communities for resource extraction.
              - Traitor: Entities betraying commitments to environmental causes, breaking promises, or siding with harmful agendas.
                - Example: Nations pledging to reduce emissions but secretly expanding fossil fuel projects.
              - Saboteur: Entities actively sabotaging systems, processes, or policies designed to protect the environment.
                - Example: Activists destroying clean energy projects or insiders derailing renewable energy initiatives.
              - Corrupt: Entities engaging in unethical or illegal activities, prioritizing personal or financial gain over environmental well-being.
                - Example: Officials accepting bribes to approve environmentally harmful projects.
              - Incompetent: Entities harming environmental efforts due to ignorance, poor decision-making, or lack of skill.
                - Example: Leaders mismanaging climate funds or implementing ineffective policies that worsen environmental crises.
              - Deceiver: Entities spreading misinformation or manipulating public perception about climate change for their benefit.
                - Example: Media outlets downplaying climate crises or corporations misleading consumers with greenwashing campaigns.


            2. Output Format:
              - Subrole: [Instigator/Conspirator/Tyrant/Traitor/Saboteur/Corrupt/Incompetent/Deceiver]
              - Just provide the label without adding anything else just the classification label


        """
    )


        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)

# Antagonist Subrole Discrimination Agent
class SubroleAgentAntagonist_UA(BaseAgent):
    def process_results(self, entity: str, news: str, location:list) -> SubroleAnalysisResult:
        """
        Discriminate between subroles of the Antagonist: Instigator, Conspirator, Tyrant, Foreign Adversary, Traitor,
        Spy, Saboteur, Corrupt, Incompetent, Terrorist, Deceiver, Bigot.
        """
        prompt = (
          f"""

          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity}based on its portrayal, actions, or impact in a news article {news}.
          of an article related to the Ukranian Conflict, you will be provided with the location where the entity is mentioned {location}.
          Choose the most adequate subrole for the Antagonist role: Instigator, Conspirator, Tyrant, Foreign Adversary, Traitor, Spy, Saboteur, Corrupt, Incompetent, Terrorist, Deceiver, or Bigot.

            Instructions:

            1. Subrole Definitions for Antagonists:
              - Instigator: Entities initiating tension, violence, or unrest in the conflict.
                - Example: Politicians using inflammatory rhetoric to incite violence or groups provoking unrest to destabilize regions.
              - Conspirator: Entities engaging in covert activities or secret plans to undermine or deceive others.
                - Example: Figures involved in espionage or secret dealings to manipulate outcomes in the conflict.
              - Tyrant: Entities abusing power to oppress others, exploit resources, or impose authoritarian control.
                - Example: Political leaders using authoritarian measures to suppress dissent or exploit civilians in occupied areas.
              - Foreign Adversary: External entities or nations acting against Ukraine’s interests, often creating geopolitical tension.
                - Example: Countries involved in military aggression or cyberattacks targeting Ukraine.
              - Traitor: Individuals betraying their country or cause by supporting adversarial actions or undermining national interests.
                - Example: Politicians or officials aiding enemy forces or leaking sensitive information.
              - Spy: Entities engaged in espionage, gathering and transmitting sensitive information to rival forces or governments.
                - Example: Individuals working covertly to leak military or strategic data.
              - Saboteur: Entities deliberately disrupting systems, infrastructure, or strategies to weaken Ukraine’s efforts.
                - Example: Insiders damaging military equipment or disrupting communications during critical operations.
              - Corrupt: Entities engaging in unethical or illegal activities, prioritizing personal or financial gain over ethical considerations.
                - Example: Officials embezzling aid funds or accepting bribes to support enemy actions.
              - Incompetent: Entities causing harm through mismanagement, ignorance, or poor decision-making.
                - Example: Leaders making disastrous strategic decisions or mishandling humanitarian efforts.
              - Terrorist: Entities engaging in acts of violence or terror to further ideological, political, or military goals.
                - Example: Groups targeting civilians with attacks to instill fear and destabilize the region.
              - Deceiver: Entities spreading misinformation, propaganda, or false narratives to manipulate public perception or gain advantage.
                - Example: Media outlets promoting false claims about the conflict or government actions.
              - Bigot: Entities engaging in discriminatory actions or rhetoric against specific groups, inciting hatred or division.
                - Example: Politicians promoting ethnic or cultural hostility to justify aggressive actions.

            2. Output Format:
              - Subrole: [Instigator/Conspirator/Tyrant/Foreign Adversary/Traitor/Spy/Saboteur/Corrupt/Incompetent/Terrorist/Deceiver/Bigot]
              - Just provide the label without adding anything else just the classification label

          """
)


        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)

# Innocent Subrole Discrimination Agent
class SubroleAgentInnocent_CC(BaseAgent):
    def process_results(self, entity: str, news: str, location:list) -> SubroleAnalysisResult:
        """
        Discriminate between subroles: Forgotten, Exploited, Victim, Scapegoat.
        """
        prompt = (
        f"""
          You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity}based on its portrayal, actions, or impact in a news article {news}.
          of an article related to Climate Change, you will be provide with the location where the entity is mentioned {location}.
          Choose the most adequate subrole for the Innocent role: Forgotten, Exploited, Victim, or Scapegoat.

            Instructions:

            1. Subrole Definitions for Innocents:
              - Forgotten: Entities portrayed as marginalized or overlooked, who do not receive the attention or support they need despite being affected by climate change.
                - Example: Indigenous populations facing discrimination and exclusion from climate discussions or displaced communities left out of international aid programs.
              - Exploited: Entities portrayed as being used for others' gain, often without consent, and with significant harm to their well-being.
                - Example: Workers subjected to poor conditions in industries contributing to deforestation or communities exploited for natural resources.
              - Victim: Entities framed as suffering harm due to circumstances beyond their control, such as physical harm (natural disasters, industrial accidents) or economic harm (poverty caused by climate-related policies).
                - Example: Farmers losing livelihoods due to droughts or victims of flooding caused by rising sea levels.
              - Scapegoat: Entities unfairly blamed for problems or failures related to climate change, often to divert attention from the real causes.
                - Example: Small communities blamed for deforestation caused by corporate interests or low-income nations held responsible for global emissions despite their minimal contribution.


            2. Output Format:
              - Subrole: [Forgotten/Exploited/Victim/Scapegoat]
              - Just provide the label without adding anything else just the classification label


        """
      )

        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)

class SubroleAgentInnocent_UA(BaseAgent):
    def process_results(self, entity: str, news: str, location:list) -> SubroleAnalysisResult:

        prompt =(

      f"""

 You are an expert in geopolitical and sociopolitical analysis.
          Your task is to classify the subrole of an entity {entity}based on its portrayal, actions, or impact in a news article {news}.
          of an article related to the Ukranian Conflict, you will be provided with the location where the entity is mentioned {location}.
          Choose the most adequate subrole for the Innocent role: Forgotten, Exploited, Victim, or Scapegoat.
          Instructions:


          1. Subrole Definitions for Innocents:
            - Forgotten: Entities portrayed as marginalized or overlooked, receiving little attention or support despite being severely affected by the conflict.
              - Example: Refugees fleeing war who lack access to aid, displaced individuals ignored by international organizations, or minority groups excluded from relief programs.
            - Exploited: Entities used for others’ gain during the conflict, often without consent and with harm to their well-being.
              - Example: Civilians forced into labor for occupying forces, local resources exploited by foreign powers, or communities coerced into serving military needs.
            - Victim: Entities suffering harm due to circumstances beyond their control, including physical harm (bombings, invasions, or attacks) or economic harm (sanctions, loss of livelihoods).
              - Example: Families losing their homes due to missile strikes, farmers unable to harvest crops due to war conditions, or civilians affected by economic blockades.
            - Scapegoat: Entities unfairly blamed for problems or failures in the conflict, often to divert attention from the real causes.
              - Example: Minority groups accused of undermining national unity, local communities wrongly blamed for aiding opposing forces, or political opponents targeted to deflect criticism.

          2. Output Format:
            - Subrole: [Forgotten/Exploited/Victim/Scapegoat]
            - Just provide the label without adding anything else just the classification label

""")

        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content
        return SubroleAnalysisResult(subrole=subrole)



In [ ]:
df_roles_infe_v21=df_roles.copy()
df_roles_infe_v22=df_roles.copy()

# 2.1 Without Majority Voting

In [ ]:
import pandas as pd
import json

# Define the process function
def process_roles_and_subroles(df_roles, df_news):
    # Initialize agents for UA and CC
    role_agent_ua = RoleAgent_UA(model="gpt-4o")
    role_agent_cc = RoleAgent_CC(model="gpt-4o")
    protagonist_agent_ua = SubroleAgentProtagonist_UA(model="gpt-4o")
    protagonist_agent_cc = SubroleAgentProtagonist_CC(model="gpt-4o")
    antagonist_agent_ua = SubroleAgentAntagonist_UA(model="gpt-4o")
    antagonist_agent_cc = SubroleAgentAntagonist_CC(model="gpt-4o")
    innocent_agent_ua = SubroleAgentInnocent_UA(model="gpt-4o")
    innocent_agent_cc = SubroleAgentInnocent_CC(model="gpt-4o")

    # Ensure columns for Role and Sub-role exist
    if 'Role' not in df_roles.columns:
        df_roles['Role'] = None
    if 'Sub-role' not in df_roles.columns:
        df_roles['Sub-role'] = None

    # Iterate over each row in df_roles
    for index, row in df_roles.iterrows():
        # Extract the necessary information
        doc_id = row['id']
        entity = row['Entity']
        location = row['Location']

        # Determine the topic from the id
        topic = "CC" if "CC" in doc_id else "UA" if "UA" in doc_id else None

        # Get the news content from df_news based on id
        news_row = df_news[df_news['id'] == doc_id]
        if news_row.empty:
            print(f"No matching news article for id: {doc_id}")
            continue  # Skip if no matching news article

        news_content = news_row['news'].values[0]


        if not topic:
            print(f"Could not determine topic for id: {doc_id}")
            continue  # Skip if the topic cannot be determined

        # Select the appropriate role agent based on the topic
        role_agent = role_agent_ua if topic == "UA" else role_agent_cc

        # Determine the role using the RoleAgent
        try:
            role_result = role_agent.process_results(entity=entity, news=news_content, location=location)
            role = role_result.role
        except Exception as e:
            print(f"Error processing role for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

        df_roles.loc[index, 'Role'] = role  # Update the Role column
        print('Role',role)

        # Select the appropriate subrole agent and process based on role
        try:
            if role == "Protagonist":
                subrole_agent = protagonist_agent_ua if topic == "UA" else protagonist_agent_cc
            elif role == "Antagonist":
                subrole_agent = antagonist_agent_ua if topic == "UA" else antagonist_agent_cc
            elif role == "Innocent":
                subrole_agent = innocent_agent_ua if topic == "UA" else innocent_agent_cc
            else:
                continue  # Skip if role is not recognized

            subrole_result = subrole_agent.process_results(entity=entity, news=news_content, location=location)
            subrole=subrole_result.subrole

            # Extract subroles list and store as JSON string
            df_roles.loc[index, 'Sub-role'] = subrole
            print('Sub-role',subrole)

        except Exception as e:
            print(f"Error processing subroles for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

    return df_roles

# Example Usage
# Assuming df_roles and df_news are already loaded as DataFrames
df_roles_infe_v12 = process_roles_and_subroles(df_roles_infe_v12, df_dev_news)

# Display the updated DataFrame
df_roles_infe_v12


In [ ]:
df_roles_infe_v12.to_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_21.csv', index=False)

In [ ]:
task1_pred_21 = pd.read_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_21.csv')

In [ ]:
results = calculate_metrics(df_annotations, df_roles_infe_v12)
print(results)

{'Main Role Accuracy': 0.7716535433070866, 'Exact Match for Subroles': 0.2204724409448819}


# 2.2 Majority Voting

In [ ]:

# Define the process function
def process_roles_and_subroles(df_roles, df_news):
    # Initialize agents for UA and CC
    role_agent_ua = RoleAgent_UA(model="gpt-4o")
    role_agent_cc = RoleAgent_CC(model="gpt-4o")
    protagonist_agent_ua = SubroleAgentProtagonist_UA(model="gpt-4o")
    protagonist_agent_cc = SubroleAgentProtagonist_CC(model="gpt-4o")
    antagonist_agent_ua = SubroleAgentAntagonist_UA(model="gpt-4o")
    antagonist_agent_cc = SubroleAgentAntagonist_CC(model="gpt-4o")
    innocent_agent_ua = SubroleAgentInnocent_UA(model="gpt-4o")
    innocent_agent_cc = SubroleAgentInnocent_CC(model="gpt-4o")

    # Ensure columns for Role and Sub-role exist
    if 'Role' not in df_roles.columns:
        df_roles['Role'] = None
    if 'Sub-role' not in df_roles.columns:
        df_roles['Sub-role'] = None

    # Iterate over each row in df_roles
    for index, row in df_roles.iterrows():
        # Extract the necessary information
        doc_id = row['id']
        entity = row['Entity']
        location = row['Location']

        # Determine the topic from the id
        topic = "CC" if "CC" in doc_id else "UA" if "UA" in doc_id else None
        if not topic:
            print(f"Could not determine topic for id: {doc_id}")
            continue  # Skip if the topic cannot be determined

        # Get the news content from df_news based on id
        news_row = df_news[df_news['id'] == doc_id]
        if news_row.empty:
            print(f"No matching news article for id: {doc_id}")
            continue  # Skip if no matching news article

        news_content = news_row['news'].values[0]

        # Select the appropriate role agent based on the topic
        role_agent = role_agent_ua if topic == "UA" else role_agent_cc

        # Determine the role using the RoleAgent (run 3 times and choose the most common)
        try:
            roles = []
            for _ in range(3):
                role_result = role_agent.process_results(entity=entity, news=news_content, location=location)
                roles.append(role_result.role)

            # Determine the most common role
            most_common_role = Counter(roles).most_common(1)[0][0]
            df_roles.loc[index, 'Role'] = most_common_role  # Update the Role column

        except Exception as e:
            print(f"Error processing role for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

        # Select the appropriate subrole agent based on role
        try:
            if most_common_role == "Protagonist":
                subrole_agent = protagonist_agent_ua if topic == "UA" else protagonist_agent_cc
            elif most_common_role == "Antagonist":
                subrole_agent = antagonist_agent_ua if topic == "UA" else antagonist_agent_cc
            elif most_common_role == "Innocent":
                subrole_agent = innocent_agent_ua if topic == "UA" else innocent_agent_cc
            else:
                continue  # Skip if role is not recognized

            # Determine subroles (run 3 times and include those appearing more than once)
            subroles = []
            for _ in range(3):
                subrole_result = subrole_agent.process_results(entity=entity, news=news_content, location=location)

                # Handle cases where subroles are returned as a string or a list
                if isinstance(subrole_result.subrole, str):
                    subroles.extend(subrole_result.subrole.split(','))  # Split subroles if they are a comma-separated string
                elif isinstance(subrole_result.subroles, list):
                    subroles.extend(subrole_result.subrole)  # Extend directly if subroles is a list

            # Debug: Print all gathered subroles for review
            print('Subroles:', subroles)

            # Filter subroles that appear more than once
            final_subrole = Counter(subroles).most_common(1)[0][0]

            # Debug: Print final selected subroles for review
            print('Final Subroles:', final_subrole)

            # Update Sub-role column as a JSON string
            df_roles.loc[index, 'Sub-role'] = final_subrole # Store as JSON string


        except Exception as e:
            print(f"Error processing subroles for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

    return df_roles

# Example Usage
# Assuming df_roles and df_news are already loaded as DataFrames
df_roles_infe_v22 = process_roles_and_subroles(df_roles_infe_v22, df_dev_news)


In [ ]:
df_roles_infe_v22.to_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_22.csv', index=False)

In [ ]:
task1_pred_22 = pd.read_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_22.csv')

In [ ]:
results = calculate_metrics(df_annotations, df_roles_infe_v22)
print(results)

{'Main Role Accuracy': 0.7244094488188977, 'Exact Match for Subroles': 0.2047244094488189}


# 3- Input Processing Non Hierarchical Signal Approach

In [ ]:
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage

# Define schema for subrole analysis result
class SubroleAnalysisResult(BaseModel):
    subrole: str = Field(description="Sub-role classification of the entity without additional information.")

# Base class for agents
class BaseAgent:
    def __init__(self, model: str, temperature: float = 0):
        self.llm = ChatOpenAI(model_name=model, temperature=temperature)

# Universal Subrole Classification Agent
class SubroleAgentUniversal(BaseAgent):
    def process_results(self, entity: str, context: str, summary: str) -> SubroleAnalysisResult:
        """
        Classifies the subrole of an entity based on its portrayal, actions, or impact in a given article.
        It includes all original subroles and provides broad applicability across different domains.
        """

        prompt = (
            f"""
            You are an expert in sociopolitical, historical, and conflict analysis.
            Your task is to classify the subrole of an entity "{entity}" based on its portrayal, actions, or impact in a news article "{summary}".
            You will be provided with the paragraph "{context}" where the entity is mentioned.

            Assign the most appropriate subrole from the following list:

            **Protagonist Subroles:**
              - **Guardian**: Protectors of values, people, or national sovereignty.
                - Example: Military forces defending their homeland, activists safeguarding the environment.
              - **Martyr**: Sacrificing oneself for a cause, evoking admiration for selflessness.
                - Example: Whistleblowers exposing corruption despite personal risk.
              - **Peacemaker**: Working toward diplomatic resolutions and reducing tensions.
                - Example: International mediators brokering peace deals.
              - **Rebel**: Challenging oppression or injustice, fighting against the status quo.
                - Example: Freedom fighters resisting occupation.
              - **Underdog**: Overcoming great odds despite disadvantages, inspiring resilience.
                - Example: Small communities standing against powerful adversaries.
              - **Virtuous**: Morally upright entities acting with integrity and fairness.
                - Example: Ethical leaders advocating for justice.

            **Antagonist Subroles:**
              - **Instigator**: Provoking unrest, conflict, or chaos.
                - Example: Political figures using inflammatory rhetoric.
              - **Conspirator**: Engaging in deception, secret plans, or manipulation.
                - Example: Governments orchestrating misinformation campaigns.
              - **Tyrant**: Abusing power to oppress or exploit others.
                - Example: Dictators suppressing freedom through authoritarian rule.
              - **Foreign Adversary**: External entities working against a nation’s interests.
                - Example: Foreign governments conducting cyberattacks.
              - **Traitor**: Betraying a cause, country, or group.
                - Example: Officials leaking classified intelligence to hostile forces.
              - **Spy**: Engaging in covert intelligence gathering or espionage.
                - Example: Undercover agents infiltrating opposition groups.
              - **Saboteur**: Deliberately disrupting systems or operations.
                - Example: Insiders damaging infrastructure or communications.
              - **Corrupt**: Engaging in unethical or illegal activities for personal gain.
                - Example: Officials embezzling funds meant for humanitarian aid.
              - **Incompetent**: Causing harm through mismanagement or poor decision-making.
                - Example: Leaders implementing disastrous policies without foresight.
              - **Terrorist**: Using violence or fear to advance ideological, political, or military goals.
                - Example: Groups targeting civilians with attacks.
              - **Deceiver**: Spreading misinformation to manipulate perceptions.
                - Example: Media outlets distorting facts for propaganda.
              - **Bigot**: Promoting discrimination, hatred, or divisive ideologies.
                - Example: Political leaders inciting hostility against certain groups.

            **Innocent Subroles:**
              - **Forgotten**: Marginalized or overlooked despite being affected.
                - Example: Displaced refugees ignored in international aid discussions.
              - **Exploited**: Used for others’ gain, often without consent.
                - Example: Workers subjected to harmful conditions in industrial sectors.
              - **Victim**: Suffering harm due to external circumstances beyond their control.
                - Example: Civilians caught in war zones losing homes and lives.
              - **Scapegoat**: Unfairly blamed for problems, diverting attention from real causes.
                - Example: Minority groups blamed for economic downturns.

            **Output Format:**
              - Subrole: [Guardian/Martyr/Peacemaker/Rebel/Underdog/Virtuous/Instigator/Conspirator/Tyrant/Foreign Adversary/Traitor/Spy/Saboteur/Corrupt/Incompetent/Terrorist/Deceiver/Bigot/Forgotten/Exploited/Victim/Scapegoat]
              - Just provide the most suitable label without adding anything else—just the classification label.
            """
        )

        response = self.llm.invoke([HumanMessage(content=prompt)])
        subrole = response.content.strip()

        return SubroleAnalysisResult(subrole=subrole)


In [ ]:
df_roles_infe_v31=df_roles.copy()
df_roles_infe_v32=df_roles.copy()

# 3.1 Majority Voting

In [ ]:
import pandas as pd
import json
from collections import Counter

# Function to extract the paragraph containing the entity
def extract_paragraph_with_entity(news_content, location):
    """
    Extract the paragraph containing the entity from the article.
    It starts from the nearest period (.) before the start location and ends at the next period (.) after the end location.
    """
    start_index = location[0]
    end_index = location[1]

    # Find the nearest period before the start index
    before = news_content[:start_index].rfind(".") + 1  # Start after the period

    # Find the nearest period after the end index
    after = news_content[end_index:].find(".")
    after = end_index + after + 1 if after != -1 else len(news_content)  # Include the period

    # Extract the paragraph and strip extra whitespace
    paragraph = news_content[before:after].strip()
    return paragraph

# Define the process function
def process_subroles(df_roles, df_news):
    # Initialize the **universal** subrole classification agent
    subrole_agent = SubroleAgentUniversal(model="gpt-4o")

    # Ensure the 'Sub-role' column exists
    if 'Sub-role' not in df_roles.columns:
        df_roles['Sub-role'] = None

    # Iterate over each row in df_roles
    for index, row in df_roles.iterrows():
        # Extract necessary information
        doc_id = row['id']
        entity = row['Entity']
        location = row['Location']

        # Get the news content from df_news
        news_row = df_news[df_news['id'] == doc_id]
        if news_row.empty:
            print(f"No matching news article for id: {doc_id}")
            continue  # Skip if no matching news article

        news_content = news_row['news'].values[0]

        # Extract context paragraph
        try:
            context = extract_paragraph_with_entity(news_content, location)
        except Exception as e:
            print(f"Error extracting context for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip if context extraction fails

        # Determine the subrole
        try:
            subroles = [subrole_agent.process_results(entity=entity, context=context, summary="").subrole for _ in range(3)]
            final_subrole = Counter(subroles).most_common(1)[0][0]

            df_roles.loc[index, 'Sub-role'] = final_subrole
        except Exception as e:
            print(f"Error processing subrole for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

    return df_roles




In [ ]:
df_roles_infe_v31_final= process_subroles(df_roles, df_dev_news)

In [ ]:
df_roles_infe_v31.to_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_31.csv', index=False)

In [ ]:
task1_pred_31 = pd.read_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_31.csv')

In [ ]:
results = calculate_metrics(df_annotations, df_roles_infe_v31)
print(results)

{'Main Role Accuracy': 0.0, 'Exact Match for Subroles': 0.3228346456692913}


# 3.2 No Majority Voting

In [ ]:
import pandas as pd
import json
from collections import Counter

# Function to extract the paragraph containing the entity
def extract_paragraph_with_entity(news_content, location):
    """
    Extract the paragraph containing the entity from the article.
    It starts from the nearest period (.) before the start location and ends at the next period (.) after the end location.
    """
    start_index, end_index = location

    # Find the nearest period before the start index
    before = news_content[:start_index].rfind(".") + 1  # Start after the period

    # Find the nearest period after the end index
    after = news_content[end_index:].find(".")
    after = end_index + after + 1 if after != -1 else len(news_content)  # Include the period

    # Extract the paragraph and strip extra whitespace
    paragraph = news_content[before:after].strip()
    return paragraph

# Define the process function
def process_subroles(df_roles, df_news):
    # Initialize the **universal** subrole classification agent
    subrole_agent = SubroleAgentUniversal(model="gpt-4o")

    # Ensure the 'Sub-role' column exists
    if 'Sub-role' not in df_roles.columns:
        df_roles['Sub-role'] = None

    # Iterate over each row in df_roles
    for index, row in df_roles.iterrows():
        # Extract necessary information
        doc_id = row['id']
        entity = row['Entity']
        location = row['Location']

        # Get the news content from df_news
        news_row = df_news[df_news['id'] == doc_id]
        if news_row.empty:
            print(f"No matching news article for id: {doc_id}")
            continue  # Skip if no matching news article

        news_content = news_row['news'].values[0]

        # Extract context paragraph
        try:
            context = extract_paragraph_with_entity(news_content, location)
        except Exception as e:
            print(f"Error extracting context for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip if context extraction fails

        # Determine the subrole using majority voting
        try:
            subroles = [subrole_agent.process_results(entity=entity, context=context, summary="").subrole for _ in range(1)]

            # Apply majority voting
            final_subrole = Counter(subroles).most_common(1)[0][0]

            # Assign the majority-voted subrole
            df_roles.at[index, 'Sub-role'] = final_subrole
        except Exception as e:
            print(f"Error processing subrole for entity '{entity}' in document '{doc_id}': {e}")
            continue  # Skip this entity on failure

    return df_roles


In [ ]:
df_roles_infe_v32= process_subroles(df_roles, df_dev_news)

In [ ]:
df_roles_infe_v32.to_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_32.csv', index=False)

In [ ]:
task1_pred_31 = pd.read_csv('/content/drive/My Drive/UNED/SemEval/EN/task1_pred_31.csv')

In [ ]:
results = calculate_metrics(df_annotations, df_roles_infe_v32)
print(results)

{'Main Role Accuracy': 0.0, 'Exact Match for Subroles': 0.33070866141732286}
